In [ ]:
import base64
import datetime
import hashlib
import json
import os
import re
import requests
from time import sleep

from requests.auth import AuthBase, HTTPBasicAuth
from requests_oauthlib import OAuth2Session

## Inspiration
The steps of generating an OAuth 2.0 token is inspired by this repo: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Bookmarks-lookup/bookmarks_lookup.py

## Twitter credentials

In [ ]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
redirect_uri = "YOUR_REDIRECT_URL"

## Set up permission scope

In [ ]:
# Set the scopes
# offline.access makes it possible to fetch 
# a new refresh token when the access token have expired
scopes = ["bookmark.read", "tweet.read", "users.read", "offline.access"]

## Create a code verifier

In [ ]:
code_verifier = base64.urlsafe_b64encode(os.urandom(30)).decode("utf-8")
code_verifier = re.sub("[^a-zA-Z0-9]+", "", code_verifier)

## Create a code challenge

In [ ]:
code_challenge = hashlib.sha256(code_verifier.encode("utf-8")).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode("utf-8")
code_challenge = code_challenge.replace("=", "")

## Start an OAuth 2.0 session

In [ ]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scopes)

## Create an authorize URL

In [ ]:
auth_url = "https://twitter.com/i/oauth2/authorize"
authorization_url, state = oauth.authorization_url(
    auth_url, code_challenge=code_challenge, code_challenge_method="S256"
)

# Visit the URL you received and authorize your app
print(authorization_url)

## Save the URL you got redirected to after authorization

In [ ]:
authorization_response = "THE_URL_YOU_GOT_REDIRECTED_TO_AFTER_AUTHORIZATION"

## Fetch your access token

In [ ]:
token_url = "https://api.twitter.com/2/oauth2/token"
auth = HTTPBasicAuth(client_id, client_secret)

In [ ]:
token = oauth.fetch_token(
    token_url=token_url,
    authorization_response=authorization_response,
    auth=auth,
    client_id=client_id,
    include_client_id=True,
    code_verifier=code_verifier,
)

In [ ]:
token

## Your access token

In [ ]:
access_token = token["access_token"]

## Make a request to the users/me endpoint to get your user ID

In [ ]:
user_me = requests.get(
    "https://api.twitter.com/2/users/me",
    headers={"Authorization": f"Bearer {access_token}"},
).json()

user_id = user_me["data"]["id"]

In [ ]:
user_id

## Make a request to the bookmarks url

In [ ]:
url = f"https://api.twitter.com/2/users/{user_id}/bookmarks"
headers = {
    "Authorization": f"Bearer {access_token}",
}
response = requests.get(url, headers=headers, params={
    'tweet.fields':'author_id,created_at',
    'expansions':'author_id',
    'user.fields':'username',
})
response.json()

In [ ]:
print("Response:\n", response.json().keys(), '\n')
print("Bookmark #1:\n", response.json()['data'][0], '\n')
print("User #1:\n", response.json()['includes']['users'][0], '\n')

## Create variable with tweets

In [ ]:
tweets = response.json()['data']

## Create mapping dict for user data

In [ ]:
user_mapping = {user['id']:user for user in response.json()['includes']['users']}

## Combine bookmarks with user data

In [ ]:
for tweet in tweets:
    tweet.update({
        'name':user_mapping[tweet['author_id']]['name'], 
        'username': user_mapping[tweet['author_id']]['username']
    })


## Optional: populate tweets with keywords with GPT-3

In [ ]:
import openai

# Open AI API key
api_key ="YOUR_OPEN_AI_GPT3_API_KEY"

for i, tweet in enumerate(tweets, start=1):

    print(f"Processing {i}/{len(tweets)} tweets", end='\r')
    
    # Create a prompt for the completion endpoint
    prompt = f"Here is a tweet, give me 5 keywords, each keyword on a new line, that describe what the tweet is about \n\n --- tweet start ---- \n\n  {tweet['text']} \n\n  --- tweet end ---:"
    
    response_gpt3 = openai.Completion.create(
      model="text-davinci-002",
      prompt=prompt,
      temperature=0.7,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    # Update tweet with keywords
    tweet.update({'keywords':response_gpt3['choices'][0]['text'].strip()})
    
    

## Add tweet to Notion database    

In [ ]:
# Get the API key from the environment variable
notion_key = "YOUR_NOTION_INTEGRATION_KEY"

# Get the database ID from the environment variable
notion_database_id = "YOUR_NOTION_DATABASE_ID"

# Set the headers
headers = {
    'Authorization': 'Bearer ' + notion_key,
    'Content-Type': 'application/json',
    'Notion-Version': '2021-08-16'
}

# Create the payload and make request
for i,tweet in enumerate(tweets, start=1):

    payload = {
        'parent': { 'database_id': notion_database_id },
        'properties': {
            'title': {
                'title': [
                    {
                        'text': {
                            'content':  tweet['username']
                        }
                    }
                ]
            },
           "Keywords": {"rich_text": [{ "type": "text", "text": { "content": tweet['keywords'] } }]},
            "Name": {"rich_text": [{ "type": "text", "text": { "content": tweet['name'] } }]},
            "Tweet": {"rich_text": [{ "type": "text", "text": { "content": tweet['text'] } }]},
            "URL": {'url': f"https://twitter.com/twitter/status/{tweet['id']}"},
            "Tweeted at": {"date": {"start": tweet['created_at'] }}

        }
    }

    # Make the request
    r = requests.post('https://api.notion.com/v1/pages', headers=headers, data=json.dumps(payload))

    # Print the response
    print(f"Tweet {i}/{len(tweets)} Response: {r.json()['object']}", end='\r')

## Get new access token

In [ ]:
refreshed_token = oauth.refresh_token(
          client_id=client_id,
          client_secret=client_secret,
          token_url=token_url,
          auth=auth,
          refresh_token=token["refresh_token"],
        )
refreshed_token

In [ ]:
access_token = refreshed_token["access_token"]